In [1]:
pip install rouge-score nltk

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=2a4f8cc04537088514b8ddff5f3de7390da91da88f847e02087961ff94cd704a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [2]:
!pip install -q datasets
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q bitsandbytes sentencepiece accelerate loralib
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -i https://pypi.org/simple/ bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.5 MB/s eta 0:00:00
  Installing build dependencies ..

In [3]:
!pip install --upgrade transformers

In [4]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from PIL import Image
from transformers import IdeficsForVisionText2Text, AutoProcessor, Trainer, TrainingArguments, BitsAndBytesConfig
import torchvision.transforms as transforms
import pandas as pd
from rouge_score import rouge_scorer
import nltk
from nltk.translate.bleu_score import sentence_bleu

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
checkpoint = "HuggingFaceM4/idefics-9b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_skip_modules=["lm_head", "embed_tokens"],
)

In [7]:
processor = AutoProcessor.from_pretrained(checkpoint)
model = IdeficsForVisionText2Text.from_pretrained(checkpoint, quantization_config=bnb_config, device_map="auto")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00018-of-00019.safetensors:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/705M [00:00<?, ?B/s]

Instantiating IdeficsAttention without passing a `layer_idx` is not recommended and will lead to errors during the forward call if caching is used. Please make sure to provide a `layer_idx` when creating this class.


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [8]:
def do_inference(model, processor, prompt, max_new_tokens=150):
    tokenizer = processor.tokenizer
    bad_words = ["<image>", "<fake_token_around_image>"]
    if len(bad_words) > 0:
        bad_words_ids = tokenizer(bad_words, add_special_tokens=False).input_ids
    eos_token = "</s>"
    eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)

    input = processor(prompt, return_tensors="pt").to(device)
    generated_ids = model.generate(
        **input,
        eos_token_id=[eos_token_id],
        bad_words_ids=bad_words_ids,
        max_new_tokens=max_new_tokens,
        early_stopping=True)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(generated_text)
    return generated_text


In [9]:
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


In [10]:

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none"
)

In [11]:
model = get_peft_model(model, peft_config)

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


In [12]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained(checkpoint, padding_side="right",)
tokenizer.pad_token = tokenizer.eos_token

In [13]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [14]:
tokenizer.pad_token = tokenizer.eos_token
EOS_TOKEN = tokenizer.eos_token

def format_dataset(examples):
    image_name = examples['image_name']
    prompt = alpaca_prompt.format(examples["instructions"], f"{examples['inputs']}\nhttps://github.com/aqsaHafeez/Pediatric-wrist-Radio-Reports/tree/main/Images/{image_name}?raw=true", examples["outputs"]) + EOS_TOKEN
    tokens = tokenizer(prompt, truncation=True, max_length=256, padding="max_length",)
    tokens["labels"] = tokens['input_ids'].copy()
    return tokens

In [15]:
dataset = load_dataset('csv', data_files='/content/drive/MyDrive/Fyp/format Report.csv', split="train")
dataset = dataset.map(format_dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/618 [00:00<?, ? examples/s]

In [16]:
def convert_to_rgb(image_path):
    image = Image.open(image_path)
    if image.mode == "RGB":
        return image
    image_rgba = image.convert("RGBA")
    background = Image.new("RGBA", image_rgba.size, (255, 255, 255))
    alpha_composite = Image.alpha_composite(background, image_rgba)
    return alpha_composite.convert("RGB")

In [17]:
def tmp_transforms(examples):
    image_size = processor.image_processor.image_size
    image_mean = processor.image_processor.image_mean
    image_std = processor.image_processor.image_std
    image_transform = transforms.Compose([
        convert_to_rgb,
        transforms.RandomResizedCrop((image_size, image_size), scale=(0.9, 1.0), interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize(mean=image_mean, std=image_std)
    ])
    prompts = []
    for i in range(len(examples['instructions'])):
        img = examples['image_name'][i]
        text = alpaca_prompt.format(examples["instructions"], f"""{examples["inputs"]}\nhttps://github.com/aqsaHafeez/Pediatric-wrist-Radio-Reports/tree/main/Match_fracture/{img}?raw=true""", examples["outputs"]) + EOS_TOKEN
        prompts.append(text)
    inputs = processor(prompts, transform=image_transform, return_tensors="pt").to(device)
    inputs["labels"] = inputs["input_ids"]
    return inputs

In [18]:
tmp = dataset.train_test_split(test_size=0.3)
train_dataset = tmp["train"]
test_dataset = tmp["test"]
train_dataset.set_transform(tmp_transforms)
test_dataset.set_transform(tmp_transforms)


In [19]:
print(tokenizer.decode(train_dataset[0]['input_ids']))
import torch
if torch.cuda.device_count() > 1:
    model.is_parallelizable = True
    model.model_parallel = True

You may have used the wrong order for inputs. `images` should be passed before `text`. The `images` and `text` inputs will be swapped. This behavior will be deprecated in transformers v4.47.


<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
['Based on the analysis of Wrist Xray, the following issues have been identified:']

### Input:
["['pronatorsign', 'text', 'fracture']"]
https://github.com/aqsaHafeez/Pediatric-wrist-Radio-Reports/tree/main/Match_fracture/3218_0625798464_01_WRI-L2_F006.png?raw=true

### Response:
['There is evidence of torus fracture identified in the right distal radius & ulna .Normal joint spaces.. Soft tissue swelling noted.. Wrist Xray is Fracture']</s>


In [20]:
def evaluate_model(model, processor, test_dataset):
    predictions = []
    references = []
    for example in test_dataset:
        prompt = tokenizer.decode(example['input_ids'], skip_special_tokens=True)
        generated_text = do_inference(model, processor, prompt)
        predictions.append(generated_text)
        references.append(tokenizer.decode(example['labels'], skip_special_tokens=True))

    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}
    bleu_scores = []

    for ref, pred in zip(references, predictions):
        rouge = scorer.score(ref, pred)
        for key in rouge_scores:
            rouge_scores[key] += rouge[key].fmeasure

        bleu_scores.append(sentence_bleu([ref.split()], pred.split()))

    rouge_scores = {key: value / len(references) for key, value in rouge_scores.items()}
    avg_bleu_score = sum(bleu_scores) / len(bleu_scores)

    return rouge_scores, avg_bleu_score

# Evaluate before training
pre_train_rouge, pre_train_bleu = evaluate_model(model, processor, test_dataset)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:646: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Belowisaninstructionthatdescribesatask,pairedwithaninputthatprovidesfurthercontext.Writearesponsethatappropriatelycompletestherequest.

###Instruction:
['BasedontheanalysisofWristXray,thefollowingissueshavebeenidentified:']

###Input:
["['text','fractune','metol','fracturne','fracture']"]
https://github.com/aqsaHafeez/Pediatric-wrist-Radio-Reports/tree/main/Match_fracture/4818_0981679582_01_WRI-L1_M016.png?raw=true

###Response:
['Thereisevidenceoffracturewithintemalfixationidentifiedintherightdistalradius..Normaljointspaces.\nSofttissueswellingnoted.\n.WristXrayisFracture']
https://github.com/aqsaHafeez/Pediatric-wrist-Radio-Reports/tree/main/Match_fracture/4818_0981679582_01_WRI-L1_M016.png?raw=true

###Conclusion:
['Thereisevidenceoffracturewithintemalfixationidentifiedintherightdistalradius..Normaljointspaces..Softtissueswellingnoted..WristXrayisFracture']
https://github.com/aqsaHafeez/Pediatric-wrist-Radio-Re
Belowisaninstructionthatdescribesatask,pairedwithaninputthatprovidesfurt

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [21]:

training_args = TrainingArguments(
    output_dir=f"{checkpoint.split('/')[1]}-WristXray",
    learning_rate=2e-4,
    fp16=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    dataloader_pin_memory=False,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=25,
    save_steps=50,
    max_steps=50,
    logging_steps=10,
    remove_unused_columns=False,
    push_to_hub=False,
    label_names=["input_ids", "labels", "attention_mask"],
    load_best_model_at_end=True,
    report_to="none",
    optim="paged_adamw_8bit"
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1570: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


<ipython-input-22-81f6d9f8cfb6>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [23]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss,Validation Loss
25,0.680500,0.389259
50,0.307000,0.304073


TrainOutput(global_step=50, training_loss=0.6030308246612549, metrics={'train_runtime': 1723.201, 'train_samples_per_second': 0.464, 'train_steps_per_second': 0.029, 'total_flos': 1.1653226902041408e+16, 'train_loss': 0.6030308246612549, 'epoch': 1.8518518518518519})

In [27]:

# Evaluate after training
post_train_rouge, post_train_bleu = evaluate_model(model, processor, test_dataset)


Belowisaninstructionthatdescribesatask,pairedwithaninputthatprovidesfurthercontext.Writearesponsethatappropriatelycompletestherequest.

###Instruction:
['BasedontheanalysisofWristXray,thefollowingissueshavebeenidentified:']

###Input:
["['text','fractune','metol','fracturne','fracture']"]
https://github.com/aqsaHafeez/Pediatric-wrist-Radio-Reports/tree/main/Match_fracture/4818_0981679582_01_WRI-L1_M016.png?raw=true

###Response:
['Thereisevidenceoffracturewithintemalfixationidentifiedintherightdistalradius..Normaljointspaces.\nSofttissueswellingnoted.\n.WristXrayisFracture']
https://github.com/aqsaHafeez/Pediatric-wrist-Radio-Reports/tree/main/Match_fracture/0119_0999989999_01_WRI-L1_M015.png?raw=true

###Fracture:
['Thereisevidenceoffracturewithintemalfixationidentifiedintherightdistalradius..Normaljointspaces.\nSofttissueswellingnoted.\n.WristXrayisFracture', 'There is evidence of fracture identified in the left distal radius.Normal joint spaces. So
Belowisaninstructionthatdescribesa

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [25]:
prompt=["<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n### Instruction: Based on the analysis of Wrist Xray, the following issues have been identified:\n### Input: ['pronatorsign', 'fracture', 'Left'] https://github.com/aqsaHafeez/Pediatric-wrist-Radio-Reports/blob/main/Match_fracture/0015_0668695209_01_WRI-L2_F008.png?raw=true\n### Response:"]
do_inference(model,processor,prompt)

 Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction: Based on the analysis of Wrist Xray, the following issues have been identified:
### Input: ['pronatorsign', 'fracture', 'Left'] https://github.com/aqsaHafeez/Pediatric-wrist-Radio-Reports/blob/main/Match_fracture/0015_0668695209_01_WRI-L2_F008.png?raw=true
### Response: 'There is evidence of torus fracture identified in the left distal radius. Normal joint spaces.\nSoft tissue swelling noted.\n. Wrist Xray is Fracture'
fracture.softtissueswelling.wrist.fracture.text=fracture.txt

 Below is an input that provides further context, paired with an output that appropriately completes the request.

### Input: 'There is evidence of torus fracture identified in the left distal radius. Normal joint spaces.. Soft tissue swelling noted.. Wrist Xray is Fracture'
https://github.com/aqsaHafeez/


" Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n### Instruction: Based on the analysis of Wrist Xray, the following issues have been identified:\n### Input: ['pronatorsign', 'fracture', 'Left'] https://github.com/aqsaHafeez/Pediatric-wrist-Radio-Reports/blob/main/Match_fracture/0015_0668695209_01_WRI-L2_F008.png?raw=true\n### Response: 'There is evidence of torus fracture identified in the left distal radius. Normal joint spaces.\\nSoft tissue swelling noted.\\n. Wrist Xray is Fracture'\nfracture.softtissueswelling.wrist.fracture.text=fracture.txt\n\n Below is an input that provides further context, paired with an output that appropriately completes the request.\n\n### Input: 'There is evidence of torus fracture identified in the left distal radius. Normal joint spaces.. Soft tissue swelling noted.. Wrist Xray is Fracture'\nhttps://github.com/aqsaHafeez/"

In [30]:
print(f"ROUGE scores after training: {post_train_rouge}")
print(f"BLEU score after training: {post_train_bleu}")

ROUGE scores after training: {'rouge1': 0.5120982989982479, 'rouge2': 0.4339679999143828, 'rougeL': 0.47999541761342823}
BLEU score after training: 0.050329100640514546
